In [1]:
!wget --random-wait -q -P data/images/non_georges -i data/non_georges.csv 
!wget --random-wait -q -P data/images/georges -i data/georges.csv 

In [3]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torch.utils.data import Dataset, DataLoader
from torchvision import utils
import copy

device = torch.device("cuda:0" if (torch.cuda.is_available()) else "cpu")

print("Device name", (torch.cuda.get_device_name() if (torch.cuda.is_available()) else "Cpu"))

torch.manual_seed(0)

device name Tesla T4


In [4]:
transform_to_input_image = transforms.Compose([
            transforms.Resize(224),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
            transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                std=[0.229, 0.224, 0.225])
])

In [5]:
train_batch_size = 64
eval_batch_size = 128

dataset = ImageFolder('data/images', transform=transform_to_input_image)

train_length = int(len(dataset) * 0.8)
val_length = len(dataset) - train_length
train_dataset, val_dataset = torch.utils.data.random_split(dataset, [train_length, val_length])

dataloaders = {}
dataloaders['train'] = DataLoader(train_dataset, batch_size=train_batch_size, shuffle=True)
dataloaders['val'] = DataLoader(val_dataset, batch_size=eval_batch_size, shuffle=True)

In [6]:
model = torchvision.models.resnet50(pretrained=True)

for param in model.parameters():
    param.requires_grad = False

model.fc = nn.Sequential(
    nn.Linear(in_features=2048, out_features=1, bias=True),
    nn.Sigmoid())
model = model.to(device)

Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [7]:
lr = 0.001
optimizer = optim.Adam(model.parameters(), lr=lr)
criterion = torch.nn.BCELoss()

In [8]:
def train_model(model, optimizer, dataloader, criterion, num_epochs=10):
    val_acc_history = []

    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)

        for phase in ['train', 'val']:
            if phase == 'train':
                model.train()  
            else:
                model.eval()  

            running_loss = 0.0
            running_corrects = 0
            
            for inputs, labels in dataloaders[phase]:
                inputs = inputs.to(device)
                labels = labels.to(device)

                optimizer.zero_grad()

                with torch.set_grad_enabled(phase == 'train'):
                    outputs = model(inputs)
                    loss = criterion(outputs.view(-1), labels.float())

                    preds = (outputs >= 0.5).int()

                    if phase == 'train':
                        loss.backward()
                        optimizer.step()

                running_loss += loss.item() * inputs.size(0)
                running_corrects += torch.sum(preds.view(-1) == labels.data)

            epoch_loss = running_loss / len(dataloaders[phase].dataset)
            epoch_acc = running_corrects.double() / len(dataloaders[phase].dataset)

            print('{} Loss: {:.4f} Acc: {:.4f}'.format(phase, epoch_loss, epoch_acc))

            if phase == 'val' and epoch_acc > best_acc:
                best_acc = epoch_acc
                best_model_wts = copy.deepcopy(model.state_dict())
            if phase == 'val':
                val_acc_history.append(epoch_acc)


    print('Best val Acc: {:4f}'.format(best_acc))

    model.load_state_dict(best_model_wts)
    return model, val_acc_history


In [10]:
best_model, acc_history = train_model(model, optimizer, dataloaders, criterion, num_epochs=5)

Epoch 0/4
----------
train Loss: 0.5524 Acc: 0.6836
val Loss: 0.4276 Acc: 0.7807
Epoch 1/4
----------
train Loss: 0.4355 Acc: 0.7864
val Loss: 0.4080 Acc: 0.8482
Epoch 2/4
----------
train Loss: 0.4048 Acc: 0.8099
val Loss: 0.3809 Acc: 0.7807
Epoch 3/4
----------
train Loss: 0.3956 Acc: 0.8072
val Loss: 0.3708 Acc: 0.8561
Epoch 4/4
----------
train Loss: 0.3989 Acc: 0.8088
val Loss: 0.3503 Acc: 0.8430
Best val Acc: 0.856140


In [11]:
torch.save(model.state_dict(), 'model.pth')